# SOS & Moments: The Subspace View

This notebook provides the code for the blog post *"SOS & Moments: The Subspace View"*. 

We will focus on the running example from the blog post: 

$$ 
\begin{align}
\hat{c} = \min_x \;&  1 + x  \\
\text{s.t.} \;& x^2 - 1 = 0
\end{align}
$$

with optimal value $\hat{c}=0$ and minimizer $\hat{x}=-1$.

In [ ]:
import matplotlib.pylab as plt
import numpy as np
from utils import smat, svec

## 1. Define (span) basis, manually

In [ ]:
phi = lambda x: np.array([1, x, x**2, x**3])

# 1. Define (span) basis, manually
B1 = phi(1.0)[:, None] @ phi(1)[None, :]
B2 = phi(-1.0)[:, None] @ phi(-1)[None, :]

## 2. Define span and nullspace bases, numerically

In [ ]:
# a. sample feasible points
x_samples = [-1.0, 1.0]

# b. construct moment matrix.
X_samples = [phi(x).reshape(-1, 1) @ phi(x).reshape(1, -1) for x in x_samples]

# Step 3: Stack the vectorized matrices into a single matrix L
L = np.array([svec(Xi) for Xi in X_samples]).T

print(f"Shape of the measurement matrix L: {L.shape}")

# Step 4: Use SVD to find the orthonormal bases
# U will contain the orthonormal basis for the column space (range)
# S will contain the singular values
# Vt is the conjugate transpose of V
U, S, Vt = np.linalg.svd(L, full_matrices=True)

# The basis for V is the set of columns in U corresponding to non-zero singular values
tol = 1e-8
rank = np.sum(S > tol)

b_vectors = U[:, :rank]

# The basis for V_perp is the set of columns in U corresponding to zero singular values
u_vectors = U[:, rank:]

print(f"Dimension of ambient space: {L.shape[0]}")
print(f"Dimension of V (span): {b_vectors.shape[1]}")
print(f"Dimension of VT (nullspace): {u_vectors.shape[1]}")

# Reshape the basis vectors back into matrices
B_basis = [smat(v) for v in b_vectors.T]
U_basis = [smat(v) for v in u_vectors.T]

In [ ]:
fig, axs = plt.subplots(b_vectors.shape[1], 1)
fig.set_size_inches(b_vectors.shape)
for i, (ax, b_i) in enumerate(zip(axs, b_vectors.T)):
    ax.matshow(b_i.reshape((1, -1)))
    ax.set_ylabel(f"$b_{i}$", visible=True)
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
fig, axs = plt.subplots(u_vectors.shape[1], 1)
fig.set_size_inches(u_vectors.shape)
for i, (ax, u_i) in enumerate(zip(axs, u_vectors.T)):
    im = ax.matshow(u_i.reshape((1, -1)))
    ax.set_ylabel(f"$u_{i}$", visible=True)
    ax.set_xticks([])
    ax.set_yticks([])

## 3. Solve problem using moment vs. SOS forms

In [ ]:
C = np.zeros((4, 4))
C[0, 0] = 1.0
C[0, 1] = 0.5
C[1, 0] = 0.5

print("Cost of feasible point x=1:", np.trace(C @ B1))
print("Cost of feasible point x=-1:", np.trace(C @ B2))

# Normalization matrix
A0 = np.zeros((4, 4))
A0[0, 0] = 1.0

In [ ]:
import cvxpy as cp
alpha = cp.Variable(len(B_basis))
objective = cp.Minimize(
    cp.sum([alpha[i] * cp.trace(C @ Bi) for i, Bi in enumerate(B_basis)])
)
constraints = [
    cp.sum([alpha[i] * Bi for i, Bi in enumerate(B_basis)]) >> 0,
    cp.sum([alpha[i] * cp.trace(A0 @ Bi) for i, Bi in enumerate(B_basis)]) == 1,
]

problem = cp.Problem(objective, constraints)
problem.solve(solver="SCS")

X = cp.sum([alpha[i].value * Bi for i, Bi in enumerate(B_basis)])

print(f"  optimal value: {problem.value:.4f}")
print(f"  alpha: {alpha.value.round(3)}")
print(f"  X:\n{X.round(3)}")

In [ ]:
c = cp.Variable()
beta = cp.Variable(len(U_basis))
objective = cp.Maximize(c)
constraints = [
    C - c * A0 + cp.sum([beta[i] * Ui for i, Ui in enumerate(U_basis)]) >> 0
]

problem = cp.Problem(objective, constraints)
problem.solve(solver="SCS", verbose=False)

H = C - problem.value * A0 + cp.sum([beta[i].value * Ui for i, Ui in enumerate(U_basis)])

print(f"  optimal value: {problem.value:.4f}")
print(f"  beta: {beta.value.round(3)}")
print(f"  H:\n{H.round(3)}")

## 4. Solve problem, using 4 different forms.

We implement the four different problem formulations in helper functions in the file sdp_formulations_cvxpy.py. 

In particular, knowing about the duality between the different formulations, we can obtain all variables $\mathbf{X}$, $\mathbf{H}$, $\mathbf{\alpha}$ and $\mathbf{\beta}$ for each of the problems. 

In [ ]:
from sdp_formulations_cvxpy import solve_sos_image, solve_sos_kernel, solve_moment_image, solve_moment_kernel

def print_sol(title, info):
    x = info["X"][0, 1]  # pick the solution value
    print(f"\n{title}")
    print(f"  optimal value: {info['c']:.4f}")
    print(f"  x: {x.round(3)}")
    print(f"  time: {info['time']*1000:.0f}ms")
    return

info_sos_image = solve_sos_image(C, A0, U_basis)
print_sol("sos image solution:", info_sos_image)
info_sos_kernel = solve_sos_kernel(C, A0, B_basis)
print_sol("sos kernel solution:", info_sos_kernel)
info_moment_image = solve_moment_image(C, A0, B_basis)
print_sol("moment image solution:", info_moment_image)
info_moment_kernel = solve_moment_kernel(C, A0, U_basis)
print_sol("moment kernel solution:", info_moment_kernel)